In [3]:
import pandas as pd
import os
import psycopg2

In [4]:
pd.set_option("display.max_rows", None)

In [19]:
chunksize = 10000
chunks = pd.read_csv('nhldata/game_plays.csv', chunksize=chunksize)
chunks

In [5]:
from config import host_name, password, user_name
param_dic = {
    'database':"nhl_shots",
    'user':user_name,
    'password':password,
    'host':host_name,
    'port':'5432'
}

In [6]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [13]:
def copy_from_file(conn, file, table):
    """
    Here we are going save the dataframe on disk as 
    a csv file, load the csv file  
    and use copy_from() to copy it to the table
    """
    # Save the dataframe to disk
    # tmp_df = "./tmp_dataframe.csv"
    # df.to_csv(tmp_df, index_label='id', header=False)
    with open(file, 'r') as f:
        cursor = conn.cursor()
        try:
            cursor.copy_from(f, table, sep=",")
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        print("copy_from_file() done")
        cursor.close()

copy_from_file(conn, 'shotsdata/shots_2010 - no_head.csv', 'shot_data')

# For Loop for all years
# n=0
# for x in range(10):
#     copy_from_file(conn, f'shotsdata/shots_201{n} - no_head.csv', 'shot_data')
#     n+=1

copy_from_file() done


In [ ]:
# Everything below here is throwaway code at the moment

In [23]:
columns = pd.read_sql("SELECT * FROM game_plays", conn).columns

In [ ]:
filenames = []
chunk_num = 0
for x in chunks:
    file = "temp_dataframe" + str(chunk_num) + ".csv"
    x[list(columns)].to_csv(file, index=False, header=False)
    filenames.append(file)
    chunk_num+=1


    

In [33]:
x.head()
# set(x.columns) ^ set(columns)

,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,dateTime,goals_away,goals_home,description,st_x,st_y
0,2016020045_1,2016020045,NaN,NaN,Game Scheduled,NaN,NaN,NaN,1,REGULAR,0,1200,2016-10-18 23:40:58,0,0,Game Scheduled,NaN,NaN
1,2016020045_2,2016020045,NaN,NaN,Period Ready,NaN,NaN,NaN,1,REGULAR,0,1200,2016-10-19 01:35:28,0,0,Period Ready,NaN,NaN
2,2016020045_3,2016020045,NaN,NaN,Period Start,NaN,NaN,NaN,1,REGULAR,0,1200,2016-10-19 01:40:50,0,0,Period Start,NaN,NaN
3,2016020045_4,2016020045,16.0,4.0,Faceoff,NaN,0.0,0.0,1,REGULAR,0,1200,2016-10-19 01:40:50,0,0,Jonathan Toews faceoff won against Claude Giroux,0.0,0.0
4,2016020045_5,2016020045,16.0,4.0,Shot,Wrist Shot,-71.0,9.0,1,REGULAR,54,1146,2016-10-19 01:41:44,0,0,Artem Anisimov Wrist Shot saved by Michal Neuv...,71.0,-9.0


In [11]:
for file in filenames:
    copy_from_file(conn, file, "game_plays")

In [23]:
with open ('temp_dataframe0.csv') as f:
    print("sup")

sup


In [ ]:
pd.read_csv("MoneyPuckDataDictionaryForPlayers.csv")

In [8]:
shots_df = pd.read_csv("shotsdata/shots_2017.csv")

In [36]:
shots_df.describe

In [6]:
import psycopg2
from config import host_name, password, user_name
connection = psycopg2.connect(
    host = host_name,
    port = 5432,
    user = user_name,
    password = password,
    database= 'nhl_shots'
    )
cursor=connection.cursor()

In [11]:
from config import host_name, password, user_name
conn = connect_to_db(host_name, 'nhl_shots', 5432, user_name, password)
curr = conn.cursor()

Connected!


In [12]:
import psycopg2
def copy_from_file(conn, df, table):
    """
    Here we are going save the dataframe on disk as 
    a csv file, load the csv file  
    and use copy_from() to copy it to the table
    """
    # Save the dataframe to disk
    tmp_df = "./tmp_dataframe.csv"
    df.to_csv(tmp_df, index_label='id', header=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    os.remove(tmp_df)

copy_from_file(conn, shots_df, 'shot_data')

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: './tmp_dataframe.csv'

In [13]:
copy_from_file(conn, shots_df, 'shot_data')

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: './tmp_dataframe.csv'

In [14]:
def execute_batch(conn, df, table, page_size=100):
    """
    Using psycopg2.extras.execute_batch() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_batch(cursor, query, tuples, page_size)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_batch() done")
    cursor.close()

execute_batch(conn, shots_df, 'shot_data')

Error: name 'extras' is not defined


1

In [17]:
shots_df.columns

Index(['shotID', 'homeTeamCode', 'awayTeamCode', 'season', 'isPlayoffGame',
       'game_id', 'homeTeamWon', 'id', 'time', 'timeUntilNextEvent',
       ...
       'xFroze', 'xRebound', 'xPlayContinuedInZone',
       'xPlayContinuedOutsideZone', 'xPlayStopped', 'xShotWasOnGoal',
       'isHomeTeam', 'shotWasOnGoal', 'teamCode', 'arenaAdjustedXCordABS'],
      dtype='object', length=124)

In [18]:
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

execute_many(conn, shots_df, 'shot_data')

Error: not all arguments converted during string formatting


1

In [21]:
import psycopg2
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

execute_values(conn, shots_df, 'shots_data')

Error: name 'extras' is not defined


1

In [20]:
list(shots_df.columns)

['shotID',
 'homeTeamCode',
 'awayTeamCode',
 'season',
 'isPlayoffGame',
 'game_id',
 'homeTeamWon',
 'id',
 'time',
 'timeUntilNextEvent',
 'timeSinceLastEvent',
 'period',
 'team',
 'location',
 'event',
 'goal',
 'shotPlayContinuedOutsideZone',
 'shotPlayContinuedInZone',
 'shotGoalieFroze',
 'shotPlayStopped',
 'shotGeneratedRebound',
 'homeTeamGoals',
 'awayTeamGoals',
 'xCord',
 'yCord',
 'xCordAdjusted',
 'yCordAdjusted',
 'shotAngle',
 'shotAngleAdjusted',
 'shotAnglePlusRebound',
 'shotAngleReboundRoyalRoad',
 'shotDistance',
 'shotType',
 'shotOnEmptyNet',
 'shotRebound',
 'shotAnglePlusReboundSpeed',
 'shotRush',
 'speedFromLastEvent',
 'lastEventxCord',
 'lastEventyCord',
 'distanceFromLastEvent',
 'lastEventShotAngle',
 'lastEventShotDistance',
 'lastEventCategory',
 'lastEventTeam',
 'homeEmptyNet',
 'awayEmptyNet',
 'homeSkatersOnIce',
 'awaySkatersOnIce',
 'awayPenalty1TimeLeft',
 'awayPenalty1Length',
 'homePenalty1TimeLeft',
 'homePenalty1Length',
 'playerPositionTha